In [3]:
%config InlineBackend.figure_format = 'svg'

In [4]:
import networkx as nx
#from node2vec import Node2Vec
import os
import pickle
import random
from itertools import chain
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from random import sample
import seaborn as sns
import numpy as np

## Functions

In [44]:
working_dir = "data/covid_literature/knowing01"
gwas_genes = pd.read_csv(working_dir + '/Kimono.AnyGWASCovid.nominal.1e-3.simplified.txt', sep="\t",header=None )

In [45]:
lit_genes =  []
lit_genes.append(gwas_genes[0].tolist())
lit_genes.sort()

In [47]:
unique_genes = {x for l in lit_genes for x in l}
len(unique_genes) #860

1256

unique_genes

with open(working_dir + '/unique_covid_genes_GWAS.txt', 'w') as f:
    for val in unique_genes:
        f.write(val)
        f.write("\n")

covid_genes = pd.read_csv(working_dir + '/unique_covid_genes_GWAS.txt', names=['covidGenes'])
covid_genes = covid_genes.sort_values(by=['covidGenes'])

# read data frame

In [48]:
final_non_gene_connections = pd.read_csv("data/covid_net/meanSim_all/meanSim_robust_nongenes.txt")

In [49]:
final_non_gene_connections

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim
0,A2M,SMTSD_Heart_Atrial_Appendage,1,gene,tissue,100,0.663943,0.741950,0.021931
1,AAGAB,SMTSD_Brain_Frontal_Cortex_BA9,1,gene,tissue,81,0.711958,0.803277,0.051792
2,AAK1,SMTSD_Brain_Anterior_cingulate_cortex_BA24,1,gene,tissue,100,0.757863,0.827443,0.034475
3,AAK1,SMTSD_Breast_Mammary_Tissue,1,gene,tissue,100,0.668527,0.741021,0.039284
4,AAK1,SMTSD_Skin_Not_Sun_Exposed_Suprapubic,1,gene,tissue,100,0.662176,0.722359,0.030489
...,...,...,...,...,...,...,...,...,...
6071,ZWILCH,SMTSD_Brain_Frontal_Cortex_BA9,1,gene,tissue,80,0.658528,0.749778,0.032053
6072,ZXDC,SMTSD_Brain_Nucleus_accumbens_basal_ganglia,1,gene,tissue,94,0.700148,0.767734,0.028239
6073,ZYG11B,SMTSD_Brain_Substantia_nigra,3,gene,tissue,91,0.696453,0.783530,0.043163
6074,ZYX,MHPNMNIA,1,gene,como,100,0.685412,0.733330,0.019922


In [50]:
final_non_gene_connections= final_non_gene_connections[final_non_gene_connections['node1'].isin(unique_genes)]

In [51]:
final_non_gene_connections

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim
17,ABCA1,AGE,1,gene,pheno,100,0.727772,0.772100,0.022543
20,ABCC2,SMTSD_Brain_Amygdala,1,gene,tissue,100,0.726440,0.756537,0.015896
21,ABCC2,SMTSD_Spleen,1,gene,tissue,100,0.677775,0.737450,0.036785
35,ABLIM2,SMTSD_Pituitary,1,gene,tissue,99,0.676106,0.728106,0.023375
36,ABLIM2,SMTSD_Esophagus_Muscularis,1,gene,tissue,89,0.664818,0.745099,0.033479
...,...,...,...,...,...,...,...,...,...
6032,ZNF618,AGE,1,gene,pheno,95,0.651148,0.714722,0.019866
6033,ZNF618,diabetesType2,3,gene,prs,82,0.651895,0.692203,0.016211
6058,ZNRF2,diabetesType2,1,gene,prs,99,0.656870,0.703664,0.028567
6059,ZNRF2,diabetesType1,1,gene,prs,98,0.655570,0.708923,0.026083


In [52]:
final_non_gene_connections["source"] = 'GWAS'

/home/icb/yue.hu/miniconda3/envs/n2v/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:

final_non_gene_connections.sort_values(by=['node1'])

,node1,node2,neighbour,type_node1,type_node2,N,mean_sim,max_sim,sem_sim,source
17,ABCA1,AGE,1,gene,pheno,100,0.727772,0.772100,0.022543,GWAS
20,ABCC2,SMTSD_Brain_Amygdala,1,gene,tissue,100,0.726440,0.756537,0.015896,GWAS
21,ABCC2,SMTSD_Spleen,1,gene,tissue,100,0.677775,0.737450,0.036785,GWAS
35,ABLIM2,SMTSD_Pituitary,1,gene,tissue,99,0.676106,0.728106,0.023375,GWAS
36,ABLIM2,SMTSD_Esophagus_Muscularis,1,gene,tissue,89,0.664818,0.745099,0.033479,GWAS
...,...,...,...,...,...,...,...,...,...,...
6032,ZNF618,AGE,1,gene,pheno,95,0.651148,0.714722,0.019866,GWAS
6033,ZNF618,diabetesType2,3,gene,prs,82,0.651895,0.692203,0.016211,GWAS
6059,ZNRF2,diabetesType1,1,gene,prs,98,0.655570,0.708923,0.026083,GWAS
6058,ZNRF2,diabetesType2,1,gene,prs,99,0.656870,0.703664,0.028567,GWAS


In [54]:
final_non_gene_connections.to_csv("results/covid_net/edgelist/GWAS_meanSim_1e3.csv")